In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing import image
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten,BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
#from keras.layers import BatchNormalization, GlobalAveragePooling2D,Dense, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
image_size = 224
batch_size = 32
epoch = 100
model_name = 'ResNet50'
DATA_DIR = 'data'

aug = ImageDataGenerator(rotation_range=45,
                        width_shift_range=0.125,
                        height_shift_range=0.125,
                        horizontal_flip=True,
                        fill_mode='nearest')

# Load dataset

In [ ]:
x = DATA_DIR+'/data_food11.npy'
y = DATA_DIR+'/label_food11.npy'
x = np.load(x)
y = np.load(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print('load data ',temp_data,' complete')

# Create model

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(image_size, image_size, 3)))
nlayers = -12

#create the model
x = base_model.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(.2)(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True
for layer in base_model.layers[:nlayers]:
    layer.trainable = False
        
opt = SGD(lr=1e-1,momentum=0.9,decay=1e-1/epoch)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
print('create model ',temp_model_name,' complete')
        


# Fit model

In [ ]:
fileModel = 'models/model_resNet50.hdf5'
checkpointer = ModelCheckpoint(filepath=fileModel, verbose=1, save_best_only=True)
        
model.fit_generator(aug.flow(X_train,y_train,batch_size=batch_size),
                    validation_data=(X_test,y_test),
                    steps_per_epoch = len(X_train)//batch_size,
                    epochs=epoch,
                    verbose=1,
                    callbacks=[checkpointer])